In [1]:
from sqlalchemy.engine import URL
from pgvector.sqlalchemy import Vector
from sqlalchemy import Integer, String
from typing import List, Optional
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import numpy as np
import sqlalchemy
from sqlalchemy import select
from datasets import load_dataset
from sqlalchemy import Float, Boolean
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pymilvus import MilvusClient
from pymilvus import FieldSchema, DataType, CollectionSchema

# download data
import os
import requests
import torch
import fitz
import json
from dotenv import load_dotenv
from google import genai
from langchain_core.prompts import PromptTemplate

## DB Connection

In [3]:
db_url = URL.create(
    drivername="postgresql+psycopg",
    username="postgres",
    password="password",
    host="localhost",
    port=5555,
    database="similarity_search_service_db",
)

## Base class for table definition

In [4]:
# Create the base class for the table definition
class Base(DeclarativeBase):
    __abstract__ = True


# Create the table definition
class Images(Base):
    __tablename__ = "images"
    VECTOR_LENGTH = 512

    # primary key
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    # image path - we will use it to store the path to the image file, after similarity search we can use it to retrieve the image and display it
    image_path: Mapped[str] = mapped_column(String(256))
    # image embedding - we will store the image embedding in this column, the image embedding is a list of 512 floats this is the output of the sentence transformer model
    image_embedding: Mapped[List[float]] = mapped_column(Vector(VECTOR_LENGTH))

In [5]:
# Connecting to the database and creating the table
engine = create_engine(db_url)
Base.metadata.create_all(engine)

## Utils

In [6]:
# reusable function to insert data into the table
def insert_image(
    engine: sqlalchemy.Engine, image_path: str, image_embedding: list[float]
):
    with Session(engine) as session:
        # create the image object
        image = Images(image_path=image_path, image_embedding=image_embedding)
        # add the image object to the session
        session.add(image)
        # commit the transaction
        session.commit()


# calculate the cosine similarity between the first image and the K rest of the images, order the images by the similarity score
def find_k_images(
    engine: sqlalchemy.Engine, k: int, orginal_image: Images
) -> list[Images]:
    with Session(engine) as session:
        # execution_options={"prebuffer_rows": True} is used to prebuffer the rows, this is useful when we want to fetch the rows in chunks and return them after session is closed
        result = session.execute(
            select(Images)
            .order_by(
                Images.image_embedding.cosine_distance(orginal_image.image_embedding)
            )
            .limit(k),
            execution_options={"prebuffer_rows": True},
        )
        return result


# find the images with the similarity score greater than 0.9
def find_images_with_similarity_score_greater_than(
    engine: sqlalchemy.Engine, similarity_score: float, orginal_image: Images
) -> list[Images]:
    with Session(engine) as session:
        result = session.execute(
            select(Images).filter(
                Images.image_embedding.cosine_distance(orginal_image.image_embedding)
                > similarity_score
            ),
            execution_options={"prebuffer_rows": True},
        )
        return result

## Insert random data

In [7]:
# insert some data into the table
N = 100
for i in range(N):
    image_path = f"image_{i}.jpg"
    image_embedding = np.random.rand(512).tolist()
    insert_image(engine, image_path, image_embedding)

# select first image from the table
with Session(engine) as session:
    image = session.query(Images).first()

In [8]:
# find the 10 most similar images to the first image
k = 10
similar_images = find_k_images(engine, k, image)
x = similar_images.fetchall()
# Extracting image paths from the result
img = x[0][0]
img.image_path

'image_0.jpg'

## Download Steam Game dataset

In [9]:
dataset = load_dataset("FronkonGames/steam-games-dataset")

# get columns names and types
columns = dataset["train"].features
print(columns)

columns_to_keep = [
    "Name",
    "Windows",
    "Linux",
    "Mac",
    "About the game",
    "Supported languages",
    "Price",
]

N = 40000
dataset = dataset["train"].select_columns(columns_to_keep).select(range(N))

{'AppID': Value('int64'), 'Name': Value('string'), 'Release date': Value('string'), 'Estimated owners': Value('string'), 'Peak CCU': Value('int64'), 'Required age': Value('int64'), 'Price': Value('float64'), 'DLC count': Value('int64'), 'About the game': Value('string'), 'Supported languages': Value('string'), 'Full audio languages': Value('string'), 'Reviews': Value('string'), 'Header image': Value('string'), 'Website': Value('string'), 'Support url': Value('string'), 'Support email': Value('string'), 'Windows': Value('bool'), 'Mac': Value('bool'), 'Linux': Value('bool'), 'Metacritic score': Value('int64'), 'Metacritic url': Value('string'), 'User score': Value('int64'), 'Positive': Value('int64'), 'Negative': Value('int64'), 'Score rank': Value('float64'), 'Achievements': Value('int64'), 'Recommendations': Value('int64'), 'Notes': Value('string'), 'Average playtime forever': Value('int64'), 'Average playtime two weeks': Value('int64'), 'Median playtime forever': Value('int64'), 'Medi

## Game table definition

In [10]:
class Games(Base):
    __tablename__ = "games"
    __table_args__ = {"extend_existing": True}

    # the vector size produced by the model taken from documentation https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2
    VECTOR_LENGTH = 512

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String(256))
    description: Mapped[str] = mapped_column(String(4096))
    windows: Mapped[bool] = mapped_column(Boolean)
    linux: Mapped[bool] = mapped_column(Boolean)
    mac: Mapped[bool] = mapped_column(Boolean)
    price: Mapped[float] = mapped_column(Float)
    game_description_embedding: Mapped[List[float]] = mapped_column(
        Vector(VECTOR_LENGTH)
    )


# Dropping and creating the table again
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

## Getting embedding model

In [41]:
checkpoint = "distiluse-base-multilingual-cased-v2"
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
model = SentenceTransformer(checkpoint, device=device)


def generate_embeddings(text: str) -> list[float]:
    return model.encode(text)

## Insert games data into database

In [12]:
def insert_games(engine, dataset):
    with tqdm(total=len(dataset)) as pbar:
        for i, game in enumerate(dataset):
            game_description = game["About the game"] or ""
            game_embedding = generate_embeddings(game_description)
            name, windows, linux, mac, price = (
                game["Name"],
                game["Windows"],
                game["Linux"],
                game["Mac"],
                game["Price"],
            )
            if name and windows and linux and mac and price and game_description:
                game = Games(
                    name=game["Name"],
                    description=game_description[0:4096],
                    windows=game["Windows"],
                    linux=game["Linux"],
                    mac=game["Mac"],
                    price=game["Price"],
                    game_description_embedding=game_embedding,
                )
                with Session(engine) as session:
                    session.add(game)
                    session.commit()
            pbar.update(1)

In [13]:
insert_games(engine, dataset)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [10:43<00:00, 62.15it/s]


In [42]:
def find_game(
    engine: sqlalchemy.Engine,
    game_description: str,
    windows: Optional[bool] = None,
    linux: Optional[bool] = None,
    mac: Optional[bool] = None,
    price: Optional[int] = None,
):
    with Session(engine) as session:
        game_embedding = generate_embeddings(game_description)

        query = select(Games).order_by(
            Games.game_description_embedding.cosine_distance(game_embedding)
        )

        if price:
            query = query.filter(Games.price <= price)
        if windows:
            query = query.filter(Games.windows == True)
        if linux:
            query = query.filter(Games.linux == True)
        if mac:
            query = query.filter(Games.mac == True)

        result = session.execute(query, execution_options={"prebuffer_rows": True})
        game = result.scalars().first()

        return game

In [81]:
game_1 = find_game(engine, "This is a game about a hero who saves the world", price=10)
print(f"Game: {game_1.name}")
print(f"Description: {game_1.description}")

game_2 = find_game(engine, game_description="Home decorating", price=20)
print(f"Game: {game_2.name}")
print(f"Description: {game_2.description}")

Game: Ultimate Spider Hero
Description: Ultimate Spider Hero game was designed for real heroes! Your mission is to help poor residents of the Metropolis and to save them from the terrible monsters. Move forward to fight your enemies and try not to fall! Features: Simple and addictive gameplay Nice graphics Awesome Ultimate Spider Hero Countless Steam achievements for you to collect! Compatibility with multiple major platforms (Windows, Mac, Linux, SteamOS) Make your way through the endless labyrinths of long, confusing city streets together with your favorite hero from countless movies and cartoons! Although this may look simple enough, things are not as easy as they seem. You will have to learn how to cling into houses properly using your web, otherwise you will fall to your demise. If you manage to do so - you will become a real superhero, armed with elusiveness, agility and speed and the ability to tirelessly swing across the rooftops and between the huge skyscrapers this urban land

In [82]:
game_3 = find_game(engine, game_description="Home decorating", mac=True, price=5)
print(f"Game: {game_3.name}")
print(f"Description: {game_3.description}")

Game: 3D PUZZLE - Old House
Description: Collect a 3D puzzle, transferring things to the right places to create a beautiful house. You need to go to the item, take it by pressing the left mouse button and take the item to the desired location marked in green. If you brought the correct item, it will snap into place and you will receive leaderboard points and achievements for this. Collect as much substance as possible as quickly as possible to get more points for the leaderboard. If you brought the wrong item, you can throw it away, it will return to the starting location so that you can pick it up again.


In [83]:
# At the begining I thought that it returns the same object
# But I see that descriptions are the same but names are different and
# that happens because of filtering
game_2.name == game_3.name, game_2.description == game_3.description

(False, True)

In [84]:
len(game_2.description), len(game_3.description)

(571, 571)

In [85]:
game = find_game(engine, game_description="Sport game", price=25)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

game = find_game(engine, game_description="Football game", price=25)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

game = find_game(engine, game_description="Sport car game", price=25)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

Game: Viki Spotter: Sports
Description: This game is great for developing your mindfulness skills. Here you have to take a good look at the pictures offered to you and find out how they differ among themselves. Whether you are an adult or a child - this may not be so easy at all, some differences are very difficult to locate for the first time. In this part of the game you have to go straight to the gym. Features: Many interesting scenes Pleasant graphics Easy gameplay Many Steam achievements The game is suitable for Windows, Mac and Linux If you suddenly forgot what the gym looks like or what kinds of sports exist around - it's time to remember this together with the little girl named Viki. She is going to implement a fascinating journey into the world of big sport and learn about all that might seem interesting. And in helping her will be specifically you. Visit various competitions and simulators, learn all about the world of sports and entertain yourself by finding differences in a

## Retrieval-Augmented Generation (RAG) service


In [26]:
# Milvus Client setup
host = "localhost"
port = "19530"

milvus_client = MilvusClient(host=host, port=port)

# loading env var
load_dotenv()

True

In [27]:
# Milvus Collection Schema definition
VECTOR_LENGTH = 768  # check the dimensionality for Silver Retriever Base (v1.1) model

id_field = FieldSchema(
    name="id", dtype=DataType.INT64, is_primary=True, description="Primary id"
)
text = FieldSchema(
    name="text", dtype=DataType.VARCHAR, max_length=4096, description="Page text"
)
embedding_text = FieldSchema(
    "embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=VECTOR_LENGTH,
    description="Embedded text",
)

fields = [id_field, text, embedding_text]

schema = CollectionSchema(
    fields=fields,
    auto_id=True,
    enable_dynamic_field=True,
    description="RAG Texts collection",
)

In [28]:
# Create Milvus Collection and index
COLLECTION_NAME = "rag_texts_and_embeddings"

milvus_client.create_collection(collection_name=COLLECTION_NAME, schema=schema)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="L2",
    params={"M": 4, "efConstruction": 64},  # lower values for speed
)

milvus_client.create_index(collection_name=COLLECTION_NAME, index_params=index_params)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME))

['rag_texts_and_embeddings']
{'collection_name': 'rag_texts_and_embeddings', 'auto_id': True, 'num_shards': 1, 'description': 'RAG Texts collection', 'fields': [{'field_id': 100, 'name': 'id', 'description': 'Primary id', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'text', 'description': 'Page text', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096}}, {'field_id': 102, 'name': 'embedding', 'description': 'Embedded text', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'functions': [], 'aliases': [], 'collection_id': 461834675364823240, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 461834685582147588}


In [62]:
# define data source and destination
## the document origin destination from which document will be downloaded
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the document
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the processed document
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"

## local destination of the embedded pages of the document
embeddings_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska-Embeddings.json"

## local destination of all above local required files
data_dir = "./lab_data"
# Creating data directory if it does not exist
os.makedirs(data_dir, exist_ok=True)

In [63]:
# Download data from the given URL
def download_pdf_data(pdf_url: str, file_name: str) -> None:
    response = requests.get(pdf_url, stream=True)
    with open(os.path.join(data_dir, file_name), "wb") as file:
        for block in response.iter_content(chunk_size=1024):
            if block:
                file.write(block)


download_pdf_data(pdf_url, file_name)

In [64]:
# Extract text from PDF and save it as JSON
def extract_pdf_text(file_name, file_json):
    document = fitz.open(os.path.join(data_dir, file_name))
    pages = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        pages.append({"page_num": page_num, "text": page_text})

    with open(os.path.join(data_dir, file_json), "w") as file:
        json.dump(pages, file, indent=4, ensure_ascii=False)


extract_pdf_text(file_name, file_json)

In [65]:
# Generate embeddings for the extracted text and save them as JSON
def generate_embeddings_milvus(file_json, embeddings_json, model):
    pages = []
    with open(os.path.join(data_dir, file_json), "r") as file:
        data = json.load(file)

    for page in data:
        pages.append(page["text"])

    embeddings = model.encode(pages)

    embeddings_paginated = []
    for page_num in range(len(embeddings)):
        embeddings_paginated.append(
            {"page_num": page_num, "embedding": embeddings[page_num].tolist()}
        )

    with open(os.path.join(data_dir, embeddings_json), "w") as file:
        json.dump(embeddings_paginated, file, indent=4, ensure_ascii=False)


model_name = "ipipan/silver-retriever-base-v1.1"
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
milvius_embedding_model = SentenceTransformer(model_name, device=device)
generate_embeddings_milvus(file_json, embeddings_json, milvius_embedding_model)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7d121b9a-fe07-4844-8a0e-33b993fd1449)')' thrown while requesting HEAD https://huggingface.co/ipipan/silver-retriever-base-v1.1/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [33]:
# Insert embeddings into Milvus collection
def insert_embeddings(file_json, embeddings_json, client=milvus_client):
    rows = []
    with (
        open(os.path.join(data_dir, file_json), "r") as t_f,
        open(os.path.join(data_dir, embeddings_json), "r") as e_f,
    ):
        text_data, embedding_data = json.load(t_f), json.load(e_f)
        text_data = list(map(lambda d: d["text"], text_data))
        embedding_data = list(map(lambda d: d["embedding"], embedding_data))

        for page, (text, embedding) in enumerate(zip(text_data, embedding_data)):
            rows.append({"text": text, "embedding": embedding})

    client.insert(collection_name="rag_texts_and_embeddings", data=rows)


insert_embeddings(file_json, embeddings_json, client=milvus_client)

# load inserted data into memory
milvus_client.load_collection("rag_texts_and_embeddings")

In [34]:
# search
def search(milvus_embedding_model, query, milvus_client=milvus_client, limit: int = 5):
    embedded_query = milvus_embedding_model.encode(query).tolist()
    result = milvus_client.search(
        collection_name="rag_texts_and_embeddings",
        data=[embedded_query],
        limit=limit,
        search_params={"metric_type": "L2"},
        output_fields=["text"],
    )
    return result


result = search(milvius_embedding_model, query="Czym jest sztuczna inteligencja")
print(result[0])

[{'id': 461834675364823495, 'distance': 29.12518882751465, 'entity': {'text': 'Historia powstania\nsztucznej inteligencji\n7\nW języku potocznym „sztuczny" oznacza to, co\njest \nwytworem \nmającym \nnaśladować \ncoś\nnaturalnego. W takim znaczeniu używamy\nterminu ,,sztuczny\'\', gdy mówimy o sztucznym\nlodowisku lub oku. Sztuczna inteligencja byłaby\nczymś (programem, maszyną) symulującym\ninteligencję naturalną, ludzką.\nSztuczna inteligencja (AI) to obszar informatyki,\nktóry skupia się na tworzeniu programów\nkomputerowych zdolnych do wykonywania\nzadań, które wymagają ludzkiej inteligencji. \nTe zadania obejmują rozpoznawanie wzorców,\nrozumienie języka naturalnego, podejmowanie\ndecyzji, uczenie się, planowanie i wiele innych.\nGłównym celem AI jest stworzenie systemów,\nktóre są zdolne do myślenia i podejmowania\ndecyzji na sposób przypominający ludzki.\nHistoria sztucznej inteligencji sięga lat 50. \nXX wieku, kiedy to powstały pierwsze koncepcje\ni modele tego, co mogłoby sta

In [35]:
# Format search output to put it into prompt as a context
def format_search_output(search_results) -> str:
    formatted_results = []
    for i, hit in enumerate(search_results[0]):
        text = hit.entity.get("text")
        formatted_results.append(f"Document: {i + 1}\nContent: {text}")
    return "\n---\n".join(formatted_results)

In [36]:
# LLM configuration
GEMINI_KEY = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_KEY)

MODEL_ID = "gemini-2.0-flash"


def generate_response(model_id, llm_client, prompt: str):
    try:
        # Send request to Gemini 2.0 Flash API and get the response
        response = llm_client.models.generate_content(
            model=model_id,
            contents=prompt,
        )
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [44]:
def build_prompt(context: str, query: str) -> str:
    # I guess prompt can be optimized more but for now it should work fine
    # E.g Anthropic models use xml tags for prompt templates, have to check out
    # what is best practice for Gemini models

    prompt = PromptTemplate.from_template(
        f"""
    INSTRUKCJE:
    1. Jesteś ekspertem ds. sztucznej inteligencji.
    2. Przeanalizuj poniższy "KONTEKST".
    3. Odpowiedz na "PYTANIE" bazując *wyłącznie* na informacjach zawartych w "KONTEKŚCIE".
    4. Jeśli "KONTEKST" nie zawiera informacji potrzebnych do odpowiedzi, napisz: "Nie posiadam wystarczających informacji w podanym kontekście, aby odpowiedzieć na to pytanie."
    5. Nie dodawaj żadnych informacji spoza "KONTEKSTU" ani nie wymyślaj odpowiedzi.
    6. Odpowiedz w języku polskim.

    KONTEKST:
    ---
    {context}
    ---

    PYTANIE:
    {query}
    """
    )
    return prompt.format(context=context, query=query)


def rag(
    llm_model, llm_client, embedding_model, query: str, milvus_client, top_k: int = 5
) -> str:
    search_results = search(
        embedding_model, query=query, milvus_client=milvus_client, limit=top_k
    )
    formatted_context = format_search_output(search_results)
    prompt = build_prompt(formatted_context, query)
    response = generate_response(llm_model, llm_client, prompt)
    return response

In [50]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Czym jest sztuczna inteligencja",
    milvus_client=milvus_client,
)
print(resp)

Sztuczna inteligencja (AI) to obszar informatyki, który skupia się na tworzeniu programów komputerowych zdolnych do wykonywania zadań, które wymagają ludzkiej inteligencji. Te zadania obejmują rozpoznawanie wzorców, rozumienie języka naturalnego, podejmowanie decyzji, uczenie się, planowanie i wiele innych. Głównym celem AI jest stworzenie systemów, które są zdolne do myślenia i podejmowania decyzji na sposób przypominający ludzki. W języku potocznym sztuczna inteligencja to coś (program, maszyna) symulujące inteligencję naturalną, ludzką.



In [51]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Co to deep learning i jakie są jego zastosowania?",
    milvus_client=milvus_client,
)
print(resp)

Uczenie głębokie to zaawansowany rodzaj uczenia maszynowego, wykorzystujący sieci algorytmów inspirowanych strukturą mózgu, zwane sieciami neuronowymi. Głęboka sieć neuronowa ma zagnieżdżone węzły neuronowe, a każde pytanie, na które odpowiada, prowadzi do zestawu powiązanych pytań. Uczenie głębokie zazwyczaj wymaga dużego zestawu danych do treningu. Zestawy treningowe do uczenia głębokiego składają się czasami z milionów punktów danych. Po wytrenowaniu głębokiej sieci neuronowej na tych dużych zestawach danych może ona lepiej poradzić sobie z niejednoznacznościami niż sieć płytka. To sprawia, że ten mechanizm jest przydatny w aplikacjach takich jak rozpoznawanie obrazu.



In [52]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Co to jest transformer i jak działa?",
    milvus_client=milvus_client,
)
print(resp)

Na podstawie podanych dokumentów, mogę stwierdzić, że Transformer jest generatywnym modelem tekstowym, który jest w stanie generować nowe teksty na podstawie podanych fragmentów lub tematów. Modele tego rodzaju (np. GPT-3) mogą pisać artykuły, opowiadania, a nawet kody programów komputerowych. Nie posiadam informacji na temat tego, jak dokładnie działa ten model.



In [57]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Jakie są narzędzia do generacji obrazów za pomocą AI?",
    milvus_client=milvus_client,
)
print(resp)

Bazując na dostarczonych informacjach, narzędzia do generowania obrazów za pomocą AI to:

*   Pix2Pix
*   Midjourney (MJ)
*   Leonardo AI
*   DALL·E 3



In [66]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Jakie są narzędzia do generacji wideo za pomocą AI?",
    milvus_client=milvus_client,
)
print(resp)

Wybrane narzędzia SI do generowania wideo: Adobe Character Animator, Animaker, Castmagic, Colourlab.Ai, CupCat, Murf.ai, RunwayML, Stable Audio, Simplified, Suno IA, Synthesia, Veed.io, Videoleap, Visla, 10Levelup.



In [42]:
resp = rag(
    llm_model=MODEL_ID,
    llm_client=gemini_client,
    embedding_model=milvius_embedding_model,
    query="Kim jest Leo Messi?",
    milvus_client=milvus_client,
)

print(resp)

Nie posiadam wystarczających informacji w podanym kontekście, aby odpowiedzieć na to pytanie.

